<a href="https://colab.research.google.com/github/BartSla/Accounting-system/blob/master/Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import os

from tensorflow.python.keras.datasets import cifar10
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.python.keras.utils import to_categorical

import numpy as np
np.random.seed(0)

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, X_test.shape

In [0]:
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

num_channels = 3

X_train = X_train.reshape(-1, img_rows, img_cols, num_channels)
X_test = X_test.reshape(-1, img_rows, img_cols, num_channels)
input_shape = (img_rows, img_cols, num_channels)

X_train.shape, X_test.shape

In [0]:
if np.max(X_train) > 1 : X_train = X_train / 255
if np.max(X_test) > 1 : X_test = X_test / 255

In [0]:
if len (y_train.shape) == 2:
  y_train = y_train.reshape(-1)
  y_tesr = y_test.reshape(-1)
  
if len (y_train.shape) == 1:
    num_classes = len(set(y_train))
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

In [0]:
model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3,3), activation = 'relu'),
    MaxPool2D (pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters=128, kernel_size=(3,3), activation = 'relu'),
    Conv2D(filters=128, kernel_size=(3,3), activation = 'relu'),
    MaxPool2D (pool_size=(2,2)),
    
    Conv2D(filters=256, kernel_size=(3,3), activation = 'relu'),
    MaxPool2D (pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(1024, activation = 'relu'),
    Dropout(0.50),
    Dense(num_classes, activation = 'softmax')
])
model.compile(loss= 'categorical_crossentropy',  optimizer='adam', metrics = ['accuracy'] )
#model.summary()

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

In [0]:
tpu_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128*8,
    verbose=2,
    validation_data=(X_test, y_test)    
)

In [0]:
score = tpu_model.evaluate(X_test, y_test, verbose=0)

In [0]:
score